# Tutorial for coronagraphy simulator tool

## Setup

- Need to clone pc branch of https://github.com/wfirst-cgi/emccd_detect
- Do pip install -e .

## Import some libraries

In [1]:
import numpy as np

import coronagraph
import detector
import util

## Set the parameters

In [2]:
cdir = 'coronagraphs/LUVOIR-A_APLC_18bw_medFPM_2021-05-07_Dyn10pm-nostaticabb/'
scene = 'scenes/10pc/-1-HIP_-TYC_SUN-mv_4.83-L_1.00-d_10.00-Teff_5778.00_inc00_zodi0001.fits'
#scene = 'scenes/10pc/-1-HIP_-TYC_SUN-mv_4.83-L_1.00-d_10.00-Teff_5778.00_inc60_zodi0100.fits'
diam = 12. # m
time = np.array([0.21304347826086958-0.1, 0.21304347826086958, 0.21304347826086958+0.1]) # yr
wave = np.array([0.5]) # micron
odir = 'tutorial/'
iwa = 8.5 # lambda/D, for LUV-A
owa = 26. # lambda/D, for LUV-A
#iwa = 2.5 # lambda/D, for LUV-B
#owa = 13. # lambda/D, for LUV-B
name = scene[12:-5]
papt = np.array([-49.77782752, 0.85732431]) # pix
rapt = 0.8 # lambda/D
fdir = 'tutorial/'
overwrite = True
Nobs = 100

## Simulate coronagraph images of science and reference target

In [3]:
coro = coronagraph.coro(cdir)

name = coro.load_scene(scene,
                       diam, # m
                       time, # yr
                       wave, # micron
                       odir)
coro.set_pang(0.) # deg

sci = coro.sim_sci(add_star=True,
                   add_plan=True,
                   add_disk=True,
                   tag='sci',
                   save_all=True)
ref = coro.sim_ref(add_star=True,
                   add_plan=False,
                   add_disk=False,
                   tag='ref',
                   save_all=True)

Initializing coronagraph...
   Radially symmetric response --> rotating PSFs (1dx)
   Fractional obscuration = 0.175
   Fractional bandpass = 0.180
   Instrument throughput = 1.000
   Coronagraph throughput = 0.318
Loading scene...
Simulating science...
Adding star...
Adding planets...
   Finished 7 of 7 planets
Adding disk...
   Loaded data cube of spatially dependent PSFs
   Finished 0 of 3 times, 0 of 1 wavelengths   14.203 s
   Finished 1 of 3 times, 0 of 1 wavelengths   2.309 s
   Finished 2 of 3 times, 0 of 1 wavelengths   0.611 s
   Finished 3 of 3 times, 1 of 1 wavelengths
Simulating reference...
Adding star...


## Convert noiseless coronagraph images into detector images

Here with photon noise, but detector noise can also be included with the det.dnimgs function.

In [4]:
det = detector.det(iwa=iwa,
                   owa=owa)

det.nlimgs(name,
           odir,
           tags=['sci_imgs', 'sci_star', 'sci_plan', 'sci_disk', 'ref_imgs'],
           overwrite=overwrite)

tint, cr_star, cr_plan, cr_disk, cr_detn = det.tint(name,
                                                    odir,
                                                    tag='sci_imgs',
                                                    papt=papt, # pix
                                                    rapt=rapt, # lambda/D
                                                    time_comp=time[1], # yr
                                                    wave_comp=wave[0], # micron
                                                    snr_targ=7.,
                                                    path_star=odir+name+'/DET/sci_star.fits',
                                                    path_plan=odir+name+'/DET/sci_plan.fits',
                                                    path_disk=odir+name+'/DET/sci_disk.fits',
                                                    detn=False,
                                                    fdir=fdir)

det.pnimgs(name,
           odir,
           tags=['sci_imgs', 'ref_imgs'],
           tint=tint, # s
           time_comp=time[1], # yr
           wave_comp=wave[0], # micron
           Nobs=Nobs,
           overwrite=overwrite)

Initializing detector...
Computing detector images...
Computing detector images...
Computing detector images...
Computing detector images...
Computing detector images...
Computing integration time...
   CR star = 2.004e-01 ph/s
   CR plan = 3.364e-01 ph/s
   CR disk = 5.415e-02 ph/s
   CR detn = 0.000e+00 ph/s
   CR nois = 8.454e-01 ph/s
Computing photon noise images...
   Frame time = 0.350 s, integration time = 366 s --> 1047 frames
Computing photon noise images...
   Frame time = 2.440 s, integration time = 366 s --> 150 frames
